In [1]:
import torch
from SpaceFlow import SpaceFlow
import scanpy as sc
import numpy as np
import pandas as pd
import os

In [2]:
path=os.getcwd() 
path

'/home/soemearth/My_Things/CS690A/Assignment_2/DLPFC1'

In [3]:
adata = sc.read_visium(path,count_file = "151507_filtered_feature_bc_matrix.h5", source_image_path = path)
adata.var_names_make_unique()

/home/soemearth/Softwear/anaconda3/envs/CG/lib/python3.11/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/soemearth/Softwear/anaconda3/envs/CG/lib/python3.11/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
sf = SpaceFlow.SpaceFlow(adata=adata)

In [5]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000, subset=True)
sc.pp.pca(adata)
spatial_locs = adata.obsm['spatial']
spatial_graph = sf.graph_alpha(spatial_locs, n_neighbors=10)
sf.adata_preprocessed = adata
sf.spatial_graph = spatial_graph

In [6]:
sf.train(spatial_regularization_strength=0.1, z_dim=50, lr=1e-3, epochs=1000, max_patience=50, min_stop=100, random_seed=42, gpu=0, regularization_acceleration=True, edge_subset_sz=1000000)

Epoch 2/1000, Loss: 1.536879062652588
Epoch 12/1000, Loss: 1.4315757751464844
Epoch 22/1000, Loss: 1.3797770738601685
Epoch 32/1000, Loss: 1.226819396018982
Epoch 42/1000, Loss: 0.8556820750236511
Epoch 52/1000, Loss: 0.5286072492599487
Epoch 62/1000, Loss: 0.35197508335113525
Epoch 72/1000, Loss: 0.23514439165592194
Epoch 82/1000, Loss: 0.21419107913970947
Epoch 92/1000, Loss: 0.15129454433918
Epoch 102/1000, Loss: 0.13310493528842926
Epoch 112/1000, Loss: 0.11648915708065033
Epoch 122/1000, Loss: 0.09582407772541046
Epoch 132/1000, Loss: 0.08600956201553345
Epoch 142/1000, Loss: 0.08509421348571777
Epoch 152/1000, Loss: 0.08369263261556625
Epoch 162/1000, Loss: 0.06661247462034225
Epoch 172/1000, Loss: 0.07093427330255508
Epoch 182/1000, Loss: 0.06934783607721329
Epoch 192/1000, Loss: 0.06368953734636307
Epoch 202/1000, Loss: 0.061856459826231
Epoch 212/1000, Loss: 0.05883663892745972
Epoch 222/1000, Loss: 0.0660877451300621
Epoch 232/1000, Loss: 0.06073514372110367
Epoch 242/1000, L

array([[ 1.1827666 ,  1.7997426 ,  0.76708144, ..., -0.6180191 ,
        -0.15317391,  0.8721873 ],
       [-0.17980361,  2.367387  ,  0.63291246, ..., -0.7995511 ,
         0.14744335,  0.20400807],
       [ 1.3976645 ,  1.5052301 ,  1.0694511 , ..., -0.6415513 ,
        -0.23907189,  0.01708648],
       ...,
       [-0.15886019,  2.5612686 ,  1.1743526 , ..., -0.66743493,
         0.55676615,  0.8856829 ],
       [-0.20165944,  1.7209781 ,  1.1384637 , ..., -0.6178266 ,
         0.3208275 , -0.11812929],
       [ 1.3115559 ,  1.7040261 ,  1.3582456 , ..., -0.5899948 ,
        -0.24275146, -0.04765297]], dtype=float32)

In [7]:
sf.segmentation(domain_label_save_filepath="./domains.csv", n_neighbors=50, resolution=1.0)

Performing domain segmentation


/home/soemearth/Softwear/anaconda3/envs/CG/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/soemearth/Softwear/anaconda3/envs/CG/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/soemearth/Softwear/anaconda3/envs/CG/lib/python3

Segmentation complete, domain labels of cells or spots saved at ./domains.csv !


In [8]:
sf.plot_segmentation(segmentation_figure_save_filepath="./domain_segmentation.pdf", colormap="tab20", scatter_sz=1., rsz=4., csz=4., wspace=.4, hspace=.5, left=0.125, right=0.9, bottom=0.1, top=0.9)

Plotting complete, segmentation figure saved at ./domain_segmentation.pdf !


In [9]:
# sf.pseudo_Spatiotemporal_Map(pSM_values_save_filepath="./pSM_values.tsv", n_neighbors=20, resolution=1.0)

Performing pseudo-Spatiotemporal Map
pseudo-Spatiotemporal Map(pSM) calculation complete, pSM values of cells or spots saved at ./pSM_values.tsv!


In [10]:
# sf.plot_pSM(pSM_figure_save_filepath="./pseudo-Spatiotemporal-Map.pdf", colormap="roma", scatter_sz=1., rsz=4., csz=4., wspace=.4, hspace=.5, left=0.125, right=0.9, bottom=0.1, top=0.9)

Plotting complete, pseudo-Spatiotemporal Map figure saved at ./pseudo-Spatiotemporal-Map.pdf !


In [11]:
array = np.loadtxt('domains.csv')
array = array.astype(int)
array = list(map(str,array))

In [12]:
adata.obs['predictions']=array

In [13]:
adata.obs['predictions']

AAACAACGAATAGTTC-1     0
AAACAAGTATCTCCCA-1     9
AAACAATCTACTAGCA-1     4
AAACACCAATAACTGC-1     6
AAACAGCTTTCAGAAG-1    15
                      ..
TTGTTGTGTGTCAAGA-1     3
TTGTTTCACATCCAGG-1     8
TTGTTTCATTAGTCTA-1     6
TTGTTTCCATACAACT-1     8
TTGTTTGTGTAAATTC-1     0
Name: predictions, Length: 4226, dtype: object

In [15]:
adata.obs.to_csv('out.csv')